# CNN

### Import necessary libraries

In [1]:
import numpy as np
import tarfile
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

### Load data and labels

In [2]:
def load_data_from_directory(data_path):
    texts = []
    labels = []
    categories = sorted(os.listdir(data_path))
    label_map = {category: idx for idx, category in enumerate(categories)}

    for category in categories:
        category_path = os.path.join(data_path, category)
        if not os.path.isdir(category_path):
            continue
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    texts.append(f.read())
                labels.append(label_map[category])
            except Exception as e:
                print(f"Cannot read file: {file_path}, error message: {e}")

    return texts, labels, categories

data_path = "./dataset/20news-18828"

# Load the dataset
texts, labels, categories = load_data_from_directory(data_path)
print(f"Loaded {len(texts)} articles.")
print(f"Number of categories: {len(categories)}, including: {categories}")

Loaded 18828 articles.
Number of categories: 20, including: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### Split data

In [3]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

### Tokenize the text data

In [4]:
vocab_size = 50000
max_length = 300

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

### Convert texts to sequences and pad sequences

In [5]:
# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

### Encode labels

In [6]:
# Encode labels as numbers
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

### Build CNN model

In [7]:
embedding_dim = 128  # Dimension of embedding layer

model = Sequential([
    Embedding(vocab_size, embedding_dim),  # Embedding layer
    Conv1D(128, 5, activation='relu'),  # 1D Convolutional layer
    GlobalMaxPooling1D(),  # Global max pooling
    Dropout(0.5),  # Dropout for regularization
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Dropout for regularization
    Dense(len(categories), activation='softmax')  # Output layer (number of categories)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [8]:
history = model.fit(
    X_train_padded, 
    y_train_encoded, 
    epochs=10, 
    batch_size=64, 
    validation_data=(X_test_padded, y_test_encoded)
)

Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 19s 70ms/step - accuracy: 0.0785 - loss: 2.9554 - val_accuracy: 0.5640 - val_loss: 1.9816
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 16s 68ms/step - accuracy: 0.4938 - loss: 1.7352 - val_accuracy: 0.8263 - val_loss: 0.7590
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.7911 - loss: 0.7713 - val_accuracy: 0.8739 - val_loss: 0.4871
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 17s 72ms/step - accuracy: 0.8943 - loss: 0.4019 - val_accuracy: 0.8959 - val_loss: 0.4155
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.9385 - loss: 0.2417 - val_accuracy: 0.9018 - val_loss: 0.3936
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.9572 - loss: 0.1666 - val_accuracy: 0.9036 - val_loss: 0.3881
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 18s 76ms/step - accuracy: 0.9699 - loss: 0.1216 - val_accuracy: 0.9063 - val_loss: 0.4046
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 18s 75ms/step - accuracy: 0.9730 - loss: 0.0999 - 

### Evaluate the model

In [9]:
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Test Accuracy: {test_accuracy * 100:.3f}%")

118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9124 - loss: 0.4238
Test Accuracy: 90.733%


### Plot training history

In [10]:
import matplotlib.pyplot as plt

In [11]:
# Plot accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.show()